In [ ]:
import pandas as pd


df = pd.read_csv("stations.csv")  # <- criando no notebook 01-alquenonas-sst (mapa)
df.head()

In [ ]:
cabo_frio = [
    "HAB_A01",
    "HAB_A02",
    "HAB_A03",
    "HAB_A04",
    "HAB_B1",
    "HAB_B2",
]


df = df[~df["Event"].isin(cabo_frio)]

In [ ]:
bbox = (
    df["LONGITUDE"].min(), df["LONGITUDE"].max(),
    df["LATITUDE"].min(), df["LATITUDE"].max()
)

In [ ]:
from oceans.datasets import woa_subset


ds = woa_subset(
    *bbox,
    variable="temperature",
    time_period="annual",
    resolution="1/4"
)

ds = ds.sel(depth=0)

In [ ]:
%%time

ds.compute()

In [ ]:
woa = {}

for k, row in df.iterrows():
    tmp = ds.sel(
        {
            "lon": row["LONGITUDE"],
            "lat": row["LATITUDE"]
        },
        method="nearest",
)
    
    woa.update(
        {
            row["Event"]:
            {
                "tmp": tmp["t_mn"].to_numpy().squeeze(),
                "lon": tmp["lon"].to_numpy(),
                "lat": tmp["lat"].to_numpy(),
                "lon_obs": row["LONGITUDE"],
                "lat_obs": row["LATITUDE"],
            }
        }
    )

In [ ]:
import seawater as sw


w = pd.DataFrame(woa).T

lats = list(zip(w["lat"], w["lat_obs"]))
lons = list(zip(w["lon"], w["lon_obs"]))
dists = [sw.dist(*location)[0] for location in zip(lats, lons)]

In [ ]:
import pandas as pd


df = pd.read_csv(
    "Habitats_sediments_Alkenone-GDGT-temperature_proxy.tab",
    skiprows=81,
    sep="\t",
)


df.head(5)

In [ ]:
pd.concat([df[["SST (1-12) [°C]", "Event"]], w.reset_index()], axis=1)